In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats

from utils import load_csv_from_folder, get_lightpath

LOG_DIR = None
SEED = 0
RNG = np.random.default_rng(seed=SEED)

In [ ]:
X, X_mean, X_sample, y_mean, y_std, y_quant, y, y_samples = load_csv_from_folder("./datasets/train/", rng=RNG)
X_test, X_mean_test, X_sample_test, y_mean_test, y_std_test, y_quant_test, y_test, y_samples_test = load_csv_from_folder("./datasets/test/", rng=RNG)

In [ ]:
X_sample_train, X_sample_val, y_samples_train, y_samples_val = train_test_split(X_sample, y_samples, random_state=SEED, test_size=0.1)

In [ ]:
gbdt = CatBoostRegressor(loss_function="RMSEWithUncertainty", iterations=1000, random_seed=SEED)
gbdt.fit(X_sample_train, y_samples_train, eval_set=(X_sample_val, y_samples_val), verbose=0);

In [ ]:
preds = gbdt.predict(X_sample)
means, stds = preds[:, 0], np.sqrt(preds[:, 1])
cdfs = stats.norm.cdf(y_samples, loc=means, scale=stds)
# Build recalibration dataset as per Eq. (5)
cal_x = np.sort(cdfs)
cal_y = (np.array(range(len(cal_x))) + 1) / len(cal_x)
# Fit isotonic regression for recalibrating the CDF and estimating quantiles
r = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip', increasing=True).fit(cal_x, cal_y)
inverse_r = IsotonicRegression(y_min=0, y_max=1, out_of_bounds='clip', increasing=True).fit(cal_y, cal_x)

plt.plot(cal_x, cal_y)
plt.xlabel("Predicted Confidence")
plt.ylabel("Expected Confidence")
plt.grid(axis="both", linewidth=0.3)
plt.xticks(ticks=np.linspace(0, 1, 11))
plt.yticks(ticks=np.linspace(0, 1, 11));

In [ ]:
preds = gbdt.predict(X_test)
means, stds = preds[:, 0], np.sqrt(preds[:, 1])
cdfs = stats.norm.cdf(y_test, loc=means, scale=stds)
cdfs_cal = r.transform(cdfs)

plot_y = []
plot_y_unc = []
p = np.linspace(0, 1, 101)
for quantile in p:
    plot_y_unc.append(np.mean(cdfs <= quantile))
    plot_y.append(np.mean(cdfs_cal <= quantile))

cal_error_uncal = np.mean(np.abs(p[1:-1] - plot_y_unc[1:-1]))
cal_error_recal = np.mean(np.abs(p[1:-1] - plot_y[1:-1]))

plt.plot(p, plot_y_unc, label="Uncalibrated (UR)")
plt.plot(p, plot_y, label="Calibrated (CR)")
plt.plot(p, p, linestyle='dashed', label="Ideal")
plt.xlabel("Expected Confidence")
plt.ylabel("Predicted Confidence")
plt.grid(axis="both", linewidth=0.3)
plt.legend()
plt.xticks(ticks=np.linspace(0, 1, 11))
plt.yticks(ticks=np.linspace(0, 1, 11))
#plt.savefig("./calibration_plot.pdf", bbox_inches="tight")

print(f"Mean Absolute Calibration Error for uncalibrated GBDT: {cal_error_uncal}")
print(f"Mean Absolute Calibration Error for recalibrated GBDT: {cal_error_recal}")

In [ ]:
# Visualize predicted CDF and PDF w.r.t. empirical
idxs_pred = np.array([0, 1800, 3600, 4400, 6200, 8000]) 
fig_cdf, axs_cdf = plt.subplots(2, 3, sharey='all', figsize=(15, 10))
fig_pdf, axs_pdf = plt.subplots(2, 3, sharey='all', figsize=(15, 10))
for i, idx_pred in enumerate(idxs_pred):
    test_data = pd.read_csv("./datasets/test/test_2.csv")
    x_pred, y_pred = get_lightpath(test_data, idx_pred)
    preds = gbdt.predict(x_pred)
    means, stds = preds[0], np.sqrt(preds[1])
    x_cdf = np.linspace(np.min(y_pred) - 5, np.max(y_pred) + 10, 1000)
    x_ecdf = np.sort(y_pred)
    y_ecdf = np.arange(len(x_ecdf))/float(len(x_ecdf))

    x_pdf = np.linspace(np.min(y_pred) - 5, np.max(y_pred) + 10, 50)
    pdf_cal_onesample = np.gradient(r.transform(stats.norm.cdf(x_pdf, loc=means, scale=stds)), x_pdf, edge_order=2)
    pdf_uncal = np.gradient(stats.norm.cdf(x_pdf, loc=means, scale=stds), x_pdf)
    pdf_uncal_onesample = np.gradient(stats.norm.cdf(x_pdf, loc=means, scale=stds), x_pdf)
    
    axs_pdf.flat[i].plot(x_pdf, pdf_uncal_onesample, label="UR")
    axs_pdf.flat[i].plot(x_pdf, pdf_cal_onesample, label="CR")
    axs_pdf.flat[i].hist(y_pred, density=True, alpha=0.4, color='grey', label="Empirical")
    fig_pdf.legend(["UR", "CR", "Empirical"], loc='upper center')

    axs_cdf.flat[i].plot(x_cdf, stats.norm.cdf(x_cdf, loc=means, scale=stds));
    axs_cdf.flat[i].plot(x_cdf, r.transform(stats.norm.cdf(x_cdf, loc=means, scale=stds)));
    axs_cdf.flat[i].grid(axis="both", linewidth=0.3)
    axs_cdf.flat[i].plot(x_ecdf, y_ecdf, label="Empirical")
    fig_cdf.legend(["UR", "CR", "Empirical"], loc='upper center')

In [ ]:
# Estimate quantiles
quintiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
cal_quintiles = inverse_r.transform(quintiles)
x = np.linspace(np.min(y_test), np.max(y_test), 10000)
preds_uncal = gbdt.predict(X_mean_test)
means_uncal, stds_uncal = preds_uncal[:, 0], np.sqrt(preds_uncal[:, 1])
rmse_unc = []
rmse_cal = []
for i, (q, q_cal) in enumerate(zip(quintiles, cal_quintiles)):
    quantiles_unc_estimates = stats.norm.ppf(q, loc=means_uncal, scale=stds_uncal)
    quantiles_estimates = stats.norm.ppf(q_cal, loc=means_uncal, scale=stds_uncal)
    print(f"Uncalibrated RMSE for {q*100}-th quantile: {np.sqrt(np.mean(np.square(np.array(quantiles_unc_estimates) - y_quant_test[:, i])))}")
    print(f"Calibrated RMSE for {q*100}-th quantile: {np.sqrt(np.mean(np.square(np.array(quantiles_estimates) - y_quant_test[:, i])))}")
    rmse_unc.append(np.sqrt(np.mean(np.square(np.array(quantiles_unc_estimates) - y_quant_test[:, i]))))
    rmse_cal.append(np.sqrt(np.mean(np.square(np.array(quantiles_estimates) - y_quant_test[:, i]))))
# quantiles_rmse = np.concatenate([np.array(rmse_unc).reshape(-1, 1), np.array(rmse_cal).reshape(-1, 1)], axis=1)
# np.savetxt(LOG_DIR + f"quantiles/{SEED}.txt", quantiles_rmse)

In [ ]:
# Estimate quantiles via quantile regression
quintiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
rmse_qr = []
for i, q in enumerate(quintiles):
    gbdt_qr = CatBoostRegressor(loss_function=f"Quantile:alpha={q}", iterations=1000, random_seed=SEED)
    gbdt_qr.fit(X_sample_train, y_samples_train, eval_set=(X_sample_val, y_samples_val), verbose=0)
    preds_qr = gbdt_qr.predict(X_mean_test)
    print(f"Quantile Regression RMSE for {q*100}-th quantile: {np.sqrt(np.mean(np.square(np.array(preds_qr) - y_quant_test[:, i])))}")
    rmse_qr.append(np.sqrt(np.mean(np.square(np.array(preds_qr) - y_quant_test[:, i]))))
# np.savetxt(LOG_DIR + f"quantile_regression/{SEED}.txt", np.array(rmse_qr))